# Loading Libraries and Data:

Importing the necessary libraries:

In [50]:
# Supress Warnings 
import warnings
warnings.filterwarnings('ignore')

# Import common GIS tools
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# For the elevation data
from scipy.spatial import cKDTree

# For the geodataframe
import geopandas as gpd
from shapely import wkt

Importing the necessary datasets:

In [51]:
# Loading in the datasets regarding elevation, shoreline, and zip codes
zip_code = pd.read_csv("C:\\Users\\jrzem\\OneDrive\\Desktop\\EY Dataset\\New York Open Source Data\\POST_OFFICE.csv")
shoreline = pd.read_csv("C:\\Users\\jrzem\\OneDrive\\Desktop\\EY Dataset\\New York Open Source Data\\NYC_Planimetric_Database__Shoreline_20250225.csv")
elevation = pd.read_csv('C:\\Users\\jrzem\\OneDrive\\Desktop\\EY Dataset\\New York Open Source Data\\Building_Elevation_and_Subgrade__BES__20250225.csv')
elevation_2 = pd.read_csv("C:\\Users\\jrzem\\OneDrive\\Desktop\\EY Dataset\\NYC_Planimetric_Database__Elevation_Points_20250301.csv")
city_map = pd.read_csv("C:\\Users\\jrzem\\OneDrive\\Desktop\\EY Dataset\\New York Open Source Data\\DCM_StreetNameChanges_Points_20250225.csv")

In [61]:
# Loading in the socioeconomic data
bronx_boroughs = pd.read_csv("C:\\Users\\jrzem\\OneDrive\\Desktop\\EY Dataset\\bronx_boroughs.csv")
manhattan_boroughs = pd.read_csv("C:\\Users\\jrzem\\OneDrive\\Desktop\\EY Dataset\\manhattan_boroughs.csv")

# Data Cleaning:

### Geographic Data:

Establishing the bounds for the data:

In [53]:
# The region that is boudning the city into the bronx and manhattan locations
lower_left = (40.75, -74.01)
upper_right = (40.88, -73.86)
# bounds = (min_lon, min_lat, max_lon, max_lat)
bounds = (lower_left[1], lower_left[0], upper_right[1], upper_right[0])

Filtering the data for the region that we are looking at:

In [54]:
# Define the bounding box
min_lat, min_lon = lower_left
max_lat, max_lon = upper_right

# Convert the_geom to a GeoDataFrame
zip_code['geometry'] = zip_code['the_geom'].apply(wkt.loads)
gdf_geometry = gpd.GeoDataFrame(zip_code, geometry='geometry')

# Filter the zip_code data points within the bounding box
filtered_zip_code = zip_code[
    (zip_code['geometry'].apply(lambda geom: geom.y) >= min_lat) &
    (zip_code['geometry'].apply(lambda geom: geom.y) <= max_lat) &
    (zip_code['geometry'].apply(lambda geom: geom.x) >= min_lon) &
    (zip_code['geometry'].apply(lambda geom: geom.x) <= max_lon)
]

# Convert the_geom to a GeoDataFrame
shoreline['geometry'] = shoreline['the_geom'].apply(wkt.loads)
gdf_geometry = gpd.GeoDataFrame(shoreline, geometry='geometry')

# Filter the shoreline data points within the bounding box
filtered_shoreline = shoreline[
    (shoreline['geometry'].apply(lambda geom: geom.bounds[1]) >= min_lat) &
    (shoreline['geometry'].apply(lambda geom: geom.bounds[3]) <= max_lat) &
    (shoreline['geometry'].apply(lambda geom: geom.bounds[0]) >= min_lon) &
    (shoreline['geometry'].apply(lambda geom: geom.bounds[2]) <= max_lon)
]

# Convert the elevation_2['Elevation'] column to meters instead of feet
elevation['z_grade'] = elevation['z_grade'] * 0.328084

# Convert the_geom to a GeoDataFrame
elevation['geometry'] = elevation['the_geom'].apply(wkt.loads)
gdf_geometry = gpd.GeoDataFrame(elevation, geometry='geometry')

# Filter the elevation data points within the bounding box
filtered_elevation = elevation[
    (elevation['latitude'] >= min_lat) &
    (elevation['latitude'] <= max_lat) &
    (elevation['longitude'] >= min_lon) &
    (elevation['longitude'] <= max_lon)
]

# Convert the_geom to a GeoDataFrame
elevation_2['geometry'] = elevation_2['the_geom'].apply(wkt.loads)
gdf_geometry = gpd.GeoDataFrame(elevation_2, geometry='geometry')

# Filter the elevation_2 data points within the bounding box
filtered_elevation_2 = elevation_2[
    (elevation_2['geometry'].apply(lambda geom: geom.y) >= min_lat) &
    (elevation_2['geometry'].apply(lambda geom: geom.y) <= max_lat) &
    (elevation_2['geometry'].apply(lambda geom: geom.x) >= min_lon) &
    (elevation_2['geometry'].apply(lambda geom: geom.x) <= max_lon)
]

# Convert the elevation_2['Elevation'] column to meters instead of feet
filtered_elevation_2['ELEVATION'] = filtered_elevation_2['ELEVATION'] * 0.328084

# Convert the_geom to a GeoDataFrame
city_map['geometry'] = city_map['the_geom'].apply(wkt.loads)
gdf_geometry = gpd.GeoDataFrame(city_map, geometry='geometry')

# Filter the city_map data points within the bounding box
filtered_city_map = city_map[
    (city_map['geometry'].apply(lambda geom: geom.y) >= min_lat) &
    (city_map['geometry'].apply(lambda geom: geom.y) <= max_lat) &
    (city_map['geometry'].apply(lambda geom: geom.x) >= min_lon) &
    (city_map['geometry'].apply(lambda geom: geom.x) <= max_lon)
]

Difference the elevation data appropriately from the z_grade and the total height from the planimetric data:

In [55]:
# Extract coordinates from filtered_elevation and filtered_elevation_2_building
coords_elevation = np.array(list(zip(filtered_elevation['longitude'], filtered_elevation['latitude'])))
coords_elevation_2 = np.array(list(zip(filtered_elevation_2['geometry'].apply(lambda geom: geom.x), filtered_elevation_2['geometry'].apply(lambda geom: geom.y))))

# Create a KDTree for filtered_elevation_2_building
tree_elevation_2 = cKDTree(coords_elevation_2)

# Query the KDTree to find the nearest neighbors in filtered_elevation_2_building for each point in filtered_elevation
distances, indices = tree_elevation_2.query(coords_elevation, k=1)

# Compute the difference in elevation
elevation_difference = filtered_elevation_2.iloc[indices]['ELEVATION'].values - filtered_elevation['z_grade'].values

# Create a new dataframe with the required columns
elevation_df = pd.DataFrame({
    'longitude': filtered_elevation['longitude'],
    'latitude': filtered_elevation['latitude'],
    'z_grade': filtered_elevation['z_grade'],
    'Total Elevation': filtered_elevation_2['ELEVATION'].values[indices],
    'Actual Elevation': elevation_difference,
    'Feat_Code': filtered_elevation_2.iloc[indices]['FEAT_CODE'].values
})

# Display the new dataframe
elevation_df.head()

,longitude,latitude,z_grade,Total Elevation,Actual Elevation,Feat_Code
25,-73.930310,40.754337,13.840880,25.827324,11.986444,3020
26,-73.926556,40.756115,14.684056,24.318522,9.634467,3020
29,-73.944877,40.809825,8.923229,22.087472,13.164243,3020
30,-73.941198,40.808671,7.995735,28.066493,20.070757,3020
33,-73.911906,40.822514,9.902231,43.235523,33.333292,3020


In [56]:
# Filter the data to only include instances where FEAT_CODE is 3020 for buildings
elevation_df_building = elevation_df[elevation_df['Feat_Code'] == 3020]
# Filter the data to only include instances where FEAT_CODE is 3010 for water
filtered_elevation_2_water = filtered_elevation_2[filtered_elevation_2['FEAT_CODE'] == 3010]
# Filter the data to only include instances where FEAT_CODE is 3000 for Spot/Bridge
filtered_elevation_2_bridge = filtered_elevation_2[filtered_elevation_2['FEAT_CODE'] == 3000]

Here are the majority datasets:
- filtered_zip_code: This includes the stations and where they are located in our region of interest using zip codes.
- filtered_shoreline: This includes the location of the shoreline given longitude and latitude.
- filtered_elevation: This gives the grade starting point of elevation that every building is at.
- filtered_elevation_2: This includes the planimetric data that houses the total elevation for buildings, water, bridges, and more.
- filtered_city_map: This showcases the interesections, corners, and squares along with where they are located in our region. 
- elevation_df: This includes the difference that gives the actual elevation of a building.
- elevation_df_building: This specifies to only select buildings.
- filtered_elevation_2_water: This specifies to only select the elevation of water that is found from the planimeric data.
- filtered_elevation_2_bridge: This specifies to only select the elevatino of bridges and others that is found from the planimetric data.

### Socioeconomic Data

In [59]:
# Define the socioeconomic column names
Sex_Age = ['Total Population','Male','Female','Under 5 years','5 to 9 years','10 to 14 years','15 to 19 years','20 to 24 years','25 to 34 years','35 to 44 years','45 to 54 years','55 to 59 years','60 to 64 years','65 to 74 years','75 to 84 years','85 years and over','Median age (years)']
Race = ['Total Population','White','Black or African American','American Indian and Alaska Native','Cherokee tribal grouping','Chippewa tribal grouping','Navajo tribal grouping','Sioux tribal grouping','Asian','Asian Idian','Chinese','Flipino','Japanese','Korean','Vietnamese','Other Asian','Native Hawaiian and Other Pacific Islander','Native Hawaiian','Guamanian or Chamorro','Samoan','Other Pacific Islander','Some other race','Hispanic or Latino (of any race)','Mexican','Puerto Rican','Cuban','Ohter Hispanic or Latino']
Housing = ['Total housing units']
Citizen = ['Citizen, 18 and over population']

Build out two clusters of data relating to the socioeconomic implications for Manhattan versus Bronx.

# Clustering:

We are hoping to cluster points by their groups of location within a certain area. This is based on the avearges, zip codes, or other features. 

Actual elevation by a certain cluster that pulls the average height of that cluster for the model in different areas.